In [ ]:
from tqdm import tqdm

In [2]:
# read in a text file
with open('nietzsche.txt', 'r') as f:
    text = f.read()

# remove weird characters
for character in "¤¦©«Ã†":
    text = text.replace(character, "")

print(f"length of text: {len(text)} characters")

length of text: 600885 characters


In [3]:
# looking at the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !"'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz
79


In [4]:
# "tokenizer" lol - character level

stoi = {ch:i for i, ch in enumerate(chars)} #string to int
itos = {i:ch for i, ch in enumerate(chars)} # int to string
decode = lambda l: ''.join(itos[i] for i in l)
encode = lambda s: [stoi[ch] for ch in s]


print(encode("what's up hello hello"))
print(decode(encode("what's up hello hello")))

[75, 60, 53, 72, 4, 71, 1, 73, 68, 1, 60, 57, 64, 64, 67, 1, 60, 57, 64, 64, 67]
what's up hello hello


In [5]:
# encoding the entire text file
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

c:\Users\willi\Documents\Projects\nanoGPT-remake\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([600885]) torch.int64
tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0, 42, 44, 39, 39, 38, 42, 32, 37,
        30,  1, 72, 60, 53, 72,  1, 43, 70, 73, 72, 60,  1, 61, 71,  1, 53,  1,
        75, 67, 65, 53, 66,  8,  8, 75, 60, 53, 72,  1, 72, 60, 57, 66, 23,  1,
        32, 71,  1, 72, 60, 57, 70, 57,  1, 66, 67, 72,  1, 59, 70, 67, 73, 66,
        56,  0, 58, 67, 70,  1, 71, 73, 71, 68, 57, 55, 72, 61, 66, 59,  1, 72,
        60, 53, 72,  1, 53, 64, 64,  1, 68, 60, 61, 64, 67, 71, 67, 68, 60, 57,
        70, 71,  7,  1, 61, 66,  1, 71, 67,  1, 58, 53, 70,  1, 53, 71,  1, 72,
        60, 57, 77,  1, 60, 53, 74, 57,  1, 54, 57, 57, 66,  0, 56, 67, 59, 65,
        53, 72, 61, 71, 72, 71,  7,  1, 60, 53, 74, 57,  1, 58, 53, 61, 64, 57,
        56,  1, 72, 67,  1, 73, 66, 56, 57, 70, 71, 72, 53, 66, 56,  1, 75, 67,
        65, 57, 66,  8,  8, 72, 60, 53, 72,  1, 72, 60, 57,  1, 72, 57, 70, 70,
        61, 54, 64, 57,  0, 71, 57, 70, 61, 67, 73, 71, 66, 57, 71, 71,  1, 53,
       

In [6]:
# split data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([39, 41, 28, 29, 24, 26, 28,  0,  0])

In [8]:

x = train_data[:block_size]
y = train_data[1:block_size+1] # offset by one (the targets)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is {context} the target is {target}")

when the input is tensor([39]) the target is 41
when the input is tensor([39, 41]) the target is 28
when the input is tensor([39, 41, 28]) the target is 29
when the input is tensor([39, 41, 28, 29]) the target is 24
when the input is tensor([39, 41, 28, 29, 24]) the target is 26
when the input is tensor([39, 41, 28, 29, 24, 26]) the target is 28
when the input is tensor([39, 41, 28, 29, 24, 26, 28]) the target is 0
when the input is tensor([39, 41, 28, 29, 24, 26, 28,  0]) the target is 0


In [9]:
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences to process in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate random positions to get blocks from
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i+1 :i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()}, the target is: {target}')

inputs:
torch.Size([4, 8])
tensor([[ 0, 65, 61, 59, 60, 72,  1, 57],
        [ 8, 53, 55, 72, 73, 53, 64, 64],
        [60, 61, 55, 60,  1, 60, 57,  1],
        [71, 60, 65, 57, 66, 72,  1, 75]])
targets:
torch.Size([4, 8])
tensor([[65, 61, 59, 60, 72,  1, 57, 74],
        [53, 55, 72, 73, 53, 64, 64, 77],
        [61, 55, 60,  1, 60, 57,  1, 60],
        [60, 65, 57, 66, 72,  1, 75, 60]])
-----
when input is [0], the target is: 65
when input is [0, 65], the target is: 61
when input is [0, 65, 61], the target is: 59
when input is [0, 65, 61, 59], the target is: 60
when input is [0, 65, 61, 59, 60], the target is: 72
when input is [0, 65, 61, 59, 60, 72], the target is: 1
when input is [0, 65, 61, 59, 60, 72, 1], the target is: 57
when input is [0, 65, 61, 59, 60, 72, 1, 57], the target is: 74
when input is [8], the target is: 53
when input is [8, 53], the target is: 55
when input is [8, 53, 55], the target is: 72
when input is [8, 53, 55, 72], the target is: 73
when input is [8, 53, 55

In [10]:
print(xb) # input to the transformer
print(yb)

tensor([[ 0, 65, 61, 59, 60, 72,  1, 57],
        [ 8, 53, 55, 72, 73, 53, 64, 64],
        [60, 61, 55, 60,  1, 60, 57,  1],
        [71, 60, 65, 57, 66, 72,  1, 75]])
tensor([[65, 61, 59, 60, 72,  1, 57, 74],
        [53, 55, 72, 73, 53, 64, 64, 77],
        [61, 55, 60,  1, 60, 57,  1, 60],
        [60, 65, 57, 66, 72,  1, 75, 60]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors of ints
        logits = self.token_embedding_table(idx) # (B,T,C)
        loss = None
        
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)

            # last time step
            logits = logits[:, -1, :] # now {B, C}

            # softmax to get probs
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) #(B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 79])
tensor(4.6560, grad_fn=<NllLossBackward0>)

D0(CHWmYgV7CiL5Q9k]MKC-?'pSZ3RvV4_rgj.4wI;
HE.sTbK?(Sd8xk.T2c6n;u-0_ykK?3PQYVIs]o-k nZ!U6fY1vDH5bhcK


# Training the model

In [12]:
# PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4293577671051025


In [14]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ees te, thechis
stawey),
s temas--atos Onound wof at of n andiouangeanalin t, sond: fathe---ke Aniod
atatism-h A
 s rt poe owe pinde, tushes fofse ngeandunof hanjestith a
owo resy pld d fis tincthets wocheio ouriolelel avequpe o cre'sid abe thend juton
itecelore t aintonoupast ong d dellondoler ir, andincan ablly  tefong, fild de ar inche-ind gus
Chenge s impoul Phiushenoulwhe at of thy w IOUUNIFTEut wiststzZin t "NANCHERurrthe soby bma on ce thalll bey; n he "the s it hit t. lath TISt"gribuppri


In [24]:
# self attention

import torch
from torch.nn import functional as F
import torch.nn as nn

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head of self-attn
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)


v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [25]:
wei


tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8019, 0.1981, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1866, 0.2450, 0.5684, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1213, 0.1005, 0.6529, 0.1253, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1295, 0.1562, 0.2298, 0.2257, 0.2587, 0.0000, 0.0000, 0.0000],
         [0.2079, 0.0302, 0.0549, 0.0984, 0.2030, 0.4056, 0.0000, 0.0000],
         [0.0634, 0.0507, 0.6594, 0.0628, 0.0176, 0.1300, 0.0161, 0.0000],
         [0.1664, 0.1406, 0.0419, 0.0267, 0.5736, 0.0095, 0.0098, 0.0317]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0407, 0.9593, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5542, 0.3894, 0.0564, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1062, 0.4853, 0.1910, 0.2175, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1472, 0.5470, 0.0707, 0.0985, 0.1366, 0.0000, 0.0000, 0.0000],
         [0.0964, 0.054